Reading any other notebook inside this notebook- In short notebook inside another notebook we. use following commands like % run

In [0]:
%run /Workspace/Databrickszero2hero/Tutorial

### To Access any external data storage like- ADLS Gen2

These values are fetch using a Service Principal in azure.

this code will only run through your own cluster not serverless coz it allows your cluster to prive access to read the info

#DB Utilities

**dbutils.fs()** to read files under particular location

[FileInfo(path='abfss://source@dlsonal.dfs.core.windows.net/Sales.csv', name='Sales.csv', size=869537, modificationTime=1751653272000)]

**dbutils.widgets()** For creating interactive input fields in notebooks. Like parameterized inputs,etc.

Sonal


**dbutils.secrets** (Secrets Management Utilities): For secure handling of sensitive information.

This scope gets generated using the Databricks crope credentials: {ADB WS URL}/#secrets/createScope
https://adb-2353204917612307.7.azuredatabricks.net/?o=2353204917612307#secrets/createScope

# DELTA LAKE

In [0]:
df.write.format('delta')\
        .mode("append")\
        .option('path', 'abfss://destination@dlsonal.dfs.core.windows.net/sales')\
        .save()
        

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

# MANAGED AND UNMANAGED DELTA TABLES

CREATING DATABASE FIRST which will be stored in our Metastore

In [0]:
%sql
CREATE CATALOG ws_adb_sonal;

org.apache.spark.sql.catalyst.analysis.CatalogAlreadyExistsException: Catalog 'ws_adb_sonal' already exists
	at com.databricks.managedcatalog.ManagedCatalogClientImpl.$anonfun$createCatalog$5(ManagedCatalogClientImpl.scala:287)
	at com.databricks.managedcatalog.ManagedCatalogClientImpl.$anonfun$recordAndWrapException$2(ManagedCatalogClientImpl.scala:4848)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at com.databricks.managedcatalog.ManagedCatalogClientImpl.$anonfun$recordAndWrapException$1(ManagedCatalogClientImpl.scala:4847)
	at com.databricks.managedcatalog.ErrorDetailsHandler.wrapServiceException(ErrorDetailsHandler.scala:31)
	at com.databricks.managedcatalog.ErrorDetailsHandler.wrapServiceException$(ErrorDetailsHandler.scala:29)
	at com.databricks.managedcatalog.ManagedCatalogClientImpl.wrapServiceException(ManagedCatalogClientImpl.scala:165)
	at com.databricks.managedcatalog.ManagedCatalogClientImpl.recordAndWrapException(ManagedCatalogClientImpl.sc

In [0]:
%sql
CREATE DATABASE salesDB;

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1315)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:1032)
	at com.databricks.logging.Usage

**Managed Table**

In [0]:
%sql
CREATE TABLE salesDB.mantable(
  ID INT,
  Name STRING,
  Marks INT
)
USING DELTA

In [0]:
%sql
INSERT INTO salesDB.mantable 
VALUES
(1, "aa", 10),
(2, "bb", 20),
(3, "cc", 30)

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
select * from salesDB.mantable;

ID,Name,Marks
1,aa,10
2,bb,20
3,cc,30


You can drop the above table which will also delete the data residing inside it.

**External Table**

Before creating your external table you must have's:
1. Role assignment created at storage level for Storage Blob Data Contributor.
2. Credential created at catalog level to access the ADLS data using Access connector for DB using it's Resource ID.
3. Creating external table at catalog level for the same credential and granting permission to it.

In [0]:
%sql
CREATE TABLE salesDB.exttable(
  ID INT,
  Name STRING,
  Marks INT
)
USING DELTA
LOCATION 'abfss://destination@dlsonal.dfs.core.windows.net/salesDB';

In [0]:
%sql
INSERT INTO salesDB.exttable 
VALUES
(1, "aa", 10),
(2, "bb", 20),
(3, "cc", 30)

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
select * from salesDB.exttable;

ID,Name,Marks
1,aa,10
2,bb,20
3,cc,30


## Delta Table Functionalities

INSERT

In [0]:
%sql
INSERT INTO salesDB.exttable 
VALUES
(4, "aa", 10),
(5, "bb", 20),
(6, "cc", 30)

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
select * from salesDB.exttable;

ID,Name,Marks
1,aa,10
2,bb,20
3,cc,30
4,aa,10
5,bb,20
6,cc,30


DELETE

In [0]:
%sql
delete from salesDB.exttable
where ID = 6;

num_affected_rows
1


In [0]:
%sql
select * from salesDB.exttable;

In [0]:
%sql
DESCRIBE HISTORY salesDB.exttable;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2025-07-29T12:31:36Z,5000556519572074,sonalchoukseyfyp@gmail.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [], batchId -> 0, auto -> true)",null,List(2361199712964802),0704-184648-hhl8rzv3,3,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 2196, p25FileSize -> 1181, numDeletionVectorsRemoved -> 1, minFileSize -> 1181, numAddedFiles -> 1, maxFileSize -> 1181, p75FileSize -> 1181, p50FileSize -> 1181, numAddedBytes -> 1181)",null,Databricks-Runtime/14.3.x-scala2.12
3,2025-07-29T12:31:24Z,5000556519572074,sonalchoukseyfyp@gmail.com,DELETE,"Map(predicate -> [""(ID#21 = 6)""])",null,List(2361199712964802),0704-184648-hhl8rzv3,2,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 11317, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 7882, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 3404)",null,Databricks-Runtime/14.3.x-scala2.12
2,2025-07-29T11:55:05Z,5000556519572074,sonalchoukseyfyp@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2361199712964802),0704-184648-hhl8rzv3,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 1098)",null,Databricks-Runtime/14.3.x-scala2.12
1,2025-07-29T11:44:46Z,5000556519572074,sonalchoukseyfyp@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2361199712964802),0704-184648-hhl8rzv3,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 1098)",null,Databricks-Runtime/14.3.x-scala2.12
0,2025-07-29T11:39:35Z,5000556519572074,sonalchoukseyfyp@gmail.com,CREATE TABLE,"Map(partitionBy -> [], description -> null, isManaged -> false, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)",null,List(2361199712964802),0704-184648-hhl8rzv3,null,WriteSerializable,true,Map(),null,Databricks-Runtime/14.3.x-scala2.12


**VESONING- TIME TRAVEL**

In [0]:
%sql
RESTORE TABLE salesDB.exttable TO VERSION AS OF 2;

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
2196,2,1,2,1181,2196


In [0]:
%sql
select * from salesDB.exttable;

ID,Name,Marks
1,aa,10
2,bb,20
3,cc,30
4,aa,10
5,bb,20
6,cc,30


**VACUUM- when we need to free up the files from the physical storage**

In [0]:
#don't run this command it's just mentioned for refrence. Since it will remove all unnecessary files required which will also finish time travelling.
# VACUUM salesDB.tablename
# Here the by default retention period is 7 days. It means we can vacuum any files less than of 7 days. Still we we want to delete it. then following is the RETAIN command for it.
# VACUUM tablename RETAIN 0 hours;


**Delta Table Optimization**

**OPTIMIZE**

In [0]:
%sql
OPTIMIZE salesDB.exttable;
-- # This will create an impact once it is used in larger tables.

path,metrics
abfss://destination@dlsonal.dfs.core.windows.net/salesDB,"List(1, 2, List(1185, 1185, 1185.0, 1, 1185), List(1098, 1098, 1098.0, 2, 2196), 0, null, 1, 2, 0, true, 0, 0, 1753986704769, 1753986722269, 4, 1, null, List(0, 0), 3, 3, 1411, 0, null)"


In [0]:
%sql
select * from salesDB.exttable

ID,Name,Marks
1,aa,10
2,bb,20
3,cc,30
4,aa,10
5,bb,20
6,cc,30


In [0]:
%sql
OPTIMIZE salesDB.exttable ZORDER BY (id);

path,metrics
abfss://destination@dlsonal.dfs.core.windows.net/salesDB,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 1185), 0, List(0, 0), 0, null), 0, 1, 1, false, 0, 0, 1753986785865, 1753986787519, 4, 0, null, List(0, 0), 3, 3, 0, 0, null)"


In [0]:
%sql
select * from salesDB.exttable

ID,Name,Marks
1,aa,10
2,bb,20
3,cc,30
4,aa,10
5,bb,20
6,cc,30


# Auto Loader using Spark Streaming

In [0]:
df = spark.readStream.format(cloudFiles)\
                     .option("cloudFiles.format", parquet)\
                     .option("cloudFiles.schemaLocation", "abfss://aldestination@dlsonal.dfs.core.windows.net/checkpoint")\
                     .Load( "abfss://alsource@dlsonal.dfs.core.windows.net/")

In [0]:
df.writeStream.format("delta")\
              .option("checkpointLocation", "abfss://aldestination@dlsonal.dfs.core.windows.net/checkpoint")\
              .Trigger(processingTime = "5 seconds")\
              .start("abfss://aldestination@dlsonal.dfs.core.windows.net/data")
              